In [2]:
#pip install implicit

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 2/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [4]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [5]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [6]:
item_features = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 2/product.csv')
item_features.head(2)

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [7]:
user_features = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 2/hh_demographic.csv')
user_features.head(2)

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [8]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [9]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [10]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [11]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [15]:
%%time

items = data_train.item_id.unique()

pop = data_train.groupby('item_id')['sales_value'].sum()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 3.37 s


In [17]:
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12729446, 907143, 2250409, 1100658, 8012664]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12381580, 829291, 916829, 1026470, 955854]"


In [21]:
# посчитаем weighted random recommendation

In [22]:
def weighted_recommendation(data, n=5):
      
    weighted = data.groupby('item_id')['quantity'].sum().reset_index()
    
    s = data.groupby('item_id')['quantity'].sum()
    w = list(x/np.sum(s) for x in s)
    
    items = weighted.item_id
    recs = np.random.choice(items, size=n, replace=False, p=w)
    #print(recs)
    return recs.tolist()

In [23]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
weighted_recs = weighted_recommendation(data_train, n=5)

result['weighted_recommendation'] = result['user_id'].apply(lambda x: weighted_recs)

Wall time: 7.09 s


In [24]:
result.head(2)

,user_id,actual,random_recommendation,weighted_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12729446, 907143, 2250409, 1100658, 8012664]","[6534178, 5703832, 6533889, 6534166, 1404121]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12381580, 829291, 916829, 1026470, 955854]","[6534178, 5703832, 6533889, 6534166, 1404121]"


In [25]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

In [26]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

Wall time: 110 ms


In [27]:
result.head(2)

,user_id,actual,random_recommendation,weighted_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12729446, 907143, 2250409, 1100658, 8012664]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12381580, 829291, 916829, 1026470, 955854]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [28]:
# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#from src.metrics import precision_at_k, recall_at_k

In [29]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [30]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.0002938295788442704

In [31]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.15523996082272082

In [32]:
result.apply(lambda row: precision_at_k(row['weighted_recommendation'], row['actual']), axis=1).mean()

0.046033300685602706

In [33]:
data.item_id.nunique()

89051

In [34]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [35]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [36]:
len(top_5000)

5000

In [37]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [38]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [39]:
data_train.item_id.nunique()

5001

In [40]:
def random_recommendation_top(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [41]:
%%time

items = top_5000

result['random_recommendation_top'] = result['user_id'].apply(lambda x: random_recommendation_top(items, n=5))

Wall time: 691 ms


In [42]:
result.head(2)

,user_id,actual,random_recommendation,weighted_recommendation,popular_recommendation,random_recommendation_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12729446, 907143, 2250409, 1100658, 8012664]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]","[957420, 9677202, 923780, 1013703, 995502]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12381580, 829291, 916829, 1026470, 955854]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]","[855718, 1110409, 862725, 920724, 1050583]"


In [43]:
result.apply(lambda row: precision_at_k(row['random_recommendation_top'], row['actual']), axis=1).mean()

0.004603330068560235

In [44]:
def weighted_recommendation_top(data, n=5):
      
    weighted = data.groupby('item_id')['quantity'].sum().reset_index()
    
    s = data.groupby('item_id')['quantity'].sum()
    w = list(x/np.sum(s) for x in s)
        
        
    items = weighted.item_id 
    recs = np.random.choice(items, size=n, replace=False, p=w)
    #print(recs)
    return recs.tolist()

In [45]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
#popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('quantity', ascending=False).head(5000)

weighted_recs = weighted_recommendation_top(top_5000, n=5)

result['weighted_recommendation_top'] = result['user_id'].apply(lambda x: weighted_recs)

Wall time: 346 ms


In [46]:
result.head(2)

,user_id,actual,random_recommendation,weighted_recommendation,popular_recommendation,random_recommendation_top,weighted_recommendation_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12729446, 907143, 2250409, 1100658, 8012664]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]","[957420, 9677202, 923780, 1013703, 995502]","[6534178, 6534166, 1404121, 6533889, 6544236]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12381580, 829291, 916829, 1026470, 955854]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]","[855718, 1110409, 862725, 920724, 1050583]","[6534178, 6534166, 1404121, 6533889, 6544236]"


In [47]:
result.apply(lambda row: precision_at_k(row['weighted_recommendation_top'], row['actual']), axis=1).mean()

0.0461312438785508

In [48]:
def popularity_recommendation_top(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['quantity'].sum().reset_index()
    #popular.sort_values('quantity', ascending=False, inplace=True)
    
    top_5000 = popular.sort_values('quantity', ascending=False).head(5000)
    
    recs = top_5000.head(n).item_id
    #print(recs)
    return recs.tolist()

In [49]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation_top(data_train, n=5)

result['popularity_recommendation_top'] = result['user_id'].apply(lambda x: popular_recs)

Wall time: 66.8 ms


In [50]:
result.head(2)

,user_id,actual,random_recommendation,weighted_recommendation,popular_recommendation,random_recommendation_top,weighted_recommendation_top,popularity_recommendation_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12729446, 907143, 2250409, 1100658, 8012664]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]","[957420, 9677202, 923780, 1013703, 995502]","[6534178, 6534166, 1404121, 6533889, 6544236]","[6534178, 6533889, 6534166, 6544236, 1404121]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12381580, 829291, 916829, 1026470, 955854]","[6534178, 5703832, 6533889, 6534166, 1404121]","[6534178, 6533889, 1029743, 6534166, 1082185]","[855718, 1110409, 862725, 920724, 1050583]","[6534178, 6534166, 1404121, 6533889, 6544236]","[6534178, 6533889, 6534166, 6544236, 1404121]"


In [51]:
result.apply(lambda row: precision_at_k(row['popularity_recommendation_top'], row['actual']), axis=1).mean()

0.0461312438785508

In [52]:
# расчет бейзлайнов по топ-5000 товаров дает качество 0,046 vs 0,00029 для random, 0,046 vs 0,046 для weighted, 0,046 vs 0,15 для popular,
# т.е. практически не меняет качество, а в некоторых случаях даже ухудшает его